# Geodesic Dome Generator - Test Notebook

This notebook tests the CadQuery-based geodesic dome generator with wedged struts.

## Test Plan
1. Generate a 2V dome at 100cm radius
2. Create just 10 struts first to verify geometry
3. Check that struts meet at vertices without gaps
4. Generate full dome if tests pass


In [1]:
import sys
sys.path.insert(0, '../src')

import cadquery as cq
# from jupyter_cadquery import show, set_defaults

# # Set default viewer options
# set_defaults(axes=True, axes0=True)
from ocp_vscode import show

In [ ]:
from chapel2.dome_generator import (
    generate_dome,
    generate_dome_struts_individually,
    generate_2v_test_dome,
    export_dome,
    generate_chapel_dome
)
from chapel2.geometry import generate_honeycomb_dome


## Step 1: Generate Geometry Data

First, let's look at the raw geometry data for a 2V dome.


In [3]:
# Generate honeycomb dome geometry at 100cm radius
radius_cm = 100.0
frequency = 2
portion = 0.5  # hemisphere

vertices, faces, edges = generate_honeycomb_dome(radius_cm, frequency, portion)

print(f"2V Honeycomb Dome at {radius_cm}cm radius:")
print(f"  Vertices: {len(vertices)}")
print(f"  Faces (windows): {len(faces)}")
print(f"  Edges (struts): {len(edges)}")
print(f"\nFirst 5 edges: {edges[:5]}")


2V Honeycomb Dome at 100.0cm radius:
  Vertices: 58
  Faces (windows): 25
  Edges (struts): 82

First 5 edges: [(55, 57), (39, 42), (40, 41), (14, 31), (28, 30)]


## Step 2: Generate 10 Test Struts

Create just 10 struts to verify the wedged strut geometry works correctly.


In [4]:
# Test parameters
strut_width = 5.0  # cm (~2 inches)
strut_depth = 5.0  # cm (~2 inches)
max_test_struts = 30

# Generate test dome with only 10 struts
test_dome, info = generate_2v_test_dome(
    radius_cm=100.0,
    strut_width=strut_width,
    strut_depth=strut_depth,
    max_struts=max_test_struts
)

print(f"Generated {len(info['edges'])} test struts")
print(f"Strut dimensions: {strut_width}cm x {strut_depth}cm")


Generated 30 test struts
Strut dimensions: 5.0cm x 5.0cm


In [5]:
# Visualize the test struts
# Note: Requires OCP CAD Viewer extension in VS Code/Cursor
# Press Cmd+Shift+P → "OCP CAD Viewer: Open Viewer" first
show(test_dome)


Using port 3939
+


## Step 3: Inspect Individual Struts

Let's look at individual struts to verify the wedged geometry is correct.


In [6]:
# Generate struts individually for inspection
strut_list, strut_info = generate_dome_struts_individually(
    radius_cm=100.0,
    frequency=2,
    portion=0.5,
    strut_width=5.0,
    strut_depth=5.0,
    dome_style="honeycomb",
    max_struts=3
)

print(f"Generated {len(strut_list)} individual struts for inspection")


Generated 3 individual struts for inspection


In [7]:
# Show first strut individually
show(strut_list[0])


## Step 4: Generate Full 2V Dome

If the test struts look correct, generate the complete 2V dome.


In [8]:
# Generate full 2V dome
full_2v_dome, full_info = generate_dome(
    radius_cm=100.0,
    frequency=2,
    portion=0.5,
    strut_width=5.0,
    strut_depth=5.0,
    dome_style="honeycomb"
)

print(f"Full 2V Dome:")
print(f"  Vertices: {full_info['num_vertices']}")
print(f"  Edges (struts): {full_info['num_edges']}")
print(f"  Faces (windows): {full_info['num_faces']}")


Full 2V Dome:
  Vertices: 58
  Edges (struts): 82
  Faces (windows): 25


In [9]:
show(full_2v_dome)


+


## Step 5: Generate 3V Chapel Dome (Full Size)

Generate the actual Chapel of MOOP dome at 8ft radius with 3V frequency.


In [3]:
# Generate the Chapel dome
# 8ft radius, 3V frequency, 2" x 2" struts
chapel_dome, chapel_info = generate_chapel_dome(
    radius_ft=8.0,
    frequency=3,
    strut_width_in=4.0,
    strut_depth_in=4.0
)

print(f"Chapel of MOOP Dome (8ft radius, 3V):")
print(f"  Vertices: {chapel_info['num_vertices']}")
print(f"  Struts: {chapel_info['num_edges']}")
print(f"  Windows: {chapel_info['num_faces']}")
print(f"  Radius: {chapel_info['radius_cm']:.1f} cm ({chapel_info['radius_cm']/30.48:.1f} ft)")
print(f"  Strut size: {chapel_info['strut_width']:.2f}cm x {chapel_info['strut_depth']:.2f}cm")


Chapel of MOOP Dome (8ft radius, 3V):
  Vertices: 198
  Struts: 286
  Windows: 89
  Radius: 243.8 cm (8.0 ft)
  Strut size: 10.16cm x 10.16cm


In [4]:
# Visualize the Chapel dome
try:
    show(chapel_dome)
except ConnectionError:
    print("⚠️  CAD Viewer not running - skipping visualization")


Using port 3939
+


## Step 5b: Compare Cuboid vs Wedged Struts

Test the new cuboid strut style - rectangular box struts instead of the tapered wedged struts.


In [15]:
# Generate a test dome with CUBOID struts
cuboid_dome, cuboid_info = generate_chapel_dome(
    radius_ft=8.0,
    frequency=3,
    strut_width_in=4.0,
    strut_depth_in=4.0,
    strut_style="cuboid"
)

print(f"Cuboid strut dome generated:")
print(f"  Struts: {len(cuboid_info['edges'])}")
print(f"  Strut style: {cuboid_info['strut_style']}")
print(f"  Strut size: {cuboid_info['strut_width']}cm x {cuboid_info['strut_depth']}cm")


Cuboid strut dome generated:
  Struts: 286
  Strut style: cuboid
  Strut size: 10.16cm x 10.16cm


In [16]:
# Visualize the cuboid strut dome
show(cuboid_dome)


+


In [5]:
# Compare individual struts - wedged vs cuboid
wedged_struts, _ = generate_dome_struts_individually(
    radius_cm=100.0, frequency=2, strut_width=5.0, strut_depth=5.0,
    strut_style="wedged", max_struts=1
)

cuboid_struts, _ = generate_dome_struts_individually(
    radius_cm=100.0, frequency=2, strut_width=5.0, strut_depth=5.0,
    strut_style="cuboid", max_struts=1
)

print("Wedged strut (left) vs Cuboid strut (right)")
# Show both struts - the cuboid will have straight sides
show(wedged_struts[0], cuboid_struts[0])


Wedged strut (left) vs Cuboid strut (right)
++


In [ ]:
from chapel2.dome_generator import generate_dome_with_hubs

# Generate the Chapel dome with hub joints
# 8ft radius, 3V frequency, 4" x 4" struts
struts_compound, hubs_compound, hub_info = generate_dome_with_hubs(
    radius_cm=8.0 * 30.48,  # 8ft in cm
    frequency=3,
    strut_width=4.0 * 2.54,  # 4" in cm
    strut_depth=4.0 * 2.54,  # 4" in cm
    dome_style="honeycomb",
    hub_inset=None  # Auto-calculate based on strut width
)

print(f"Hub Joint Dome Generated:")
print(f"  Struts: {hub_info['num_struts_generated']}")
print(f"  Hubs: {hub_info['num_hubs_generated']}")
print(f"  Hub inset: {hub_info['hub_inset']:.1f} cm ({hub_info['hub_inset']/2.54:.1f} in)")
print(f"  Strut width: {hub_info['strut_width']:.1f} cm ({hub_info['strut_width']/2.54:.1f} in)")
print(f"  Strut depth: {hub_info['strut_depth']:.1f} cm ({hub_info['strut_depth']/2.54:.1f} in)")


Hub Joint Dome Generated:
  Struts: 286
  Hubs: 198
  Hub inset: 3.6 cm (1.4 in)
  Strut width: 10.2 cm (4.0 in)
  Strut depth: 10.2 cm (4.0 in)


In [8]:
# Visualize struts and hubs together
# Struts and hubs shown together (different colors in OCP viewer)
show(struts_compound, hubs_compound)


++


In [12]:
# Generate a smaller subset to inspect hub joints up close
struts_small, hubs_small, _ = generate_dome_with_hubs(
    radius_cm=100.0,
    frequency=2,
    strut_width=5.0,
    strut_depth=5.0,
    dome_style="honeycomb",
    max_struts=15  # Just a few struts to see the hub details
)

print(f"Close-up view: {len(list(struts_small.Solids()))} struts, {len(list(hubs_small.Solids()))} hubs")
show(struts_small, hubs_small)


Close-up view: 15 struts, 24 hubs
cc


In [9]:
# View just the hubs to see their shapes
print("Hub shapes only (without struts):")
show(hubs_small)


Hub shapes only (without struts):
+


In [10]:
# Analyze hub statistics
from collections import Counter

hub_strut_counts = [h['num_struts'] for h in hub_info['hub_infos']]
count_distribution = Counter(hub_strut_counts)

print("Hub Statistics:")
print(f"  Total hubs: {len(hub_info['hub_infos'])}")
print(f"\nStruts per hub distribution:")
for num_struts, count in sorted(count_distribution.items()):
    print(f"  {num_struts} struts: {count} hubs")


Hub Statistics:
  Total hubs: 198

Struts per hub distribution:
  2 struts: 22 hubs
  3 struts: 176 hubs


## Step 6: Export Files

Export the dome to STEP and STL files for CAD and 3D printing.


In [6]:
# Export the Chapel dome
export_dome(
    chapel_dome,
    step_path="../output/chapel_dome_3v.step",
    stl_path="../output/chapel_dome_3v.stl"
)


NameError: name 'chapel_dome' is not defined

## Analysis: Window Sizes and Cut List

Generate assembly information for the Chapel dome.


In [ ]:
from chapel2.geometry import analyze_polygon_sizes, generate_assembly_guide, format_assembly_guide_text

# Analyze window sizes
vertices = chapel_info['vertices']
faces = chapel_info['faces']
edges = chapel_info['edges']

sizes = analyze_polygon_sizes(vertices, faces)

if 'hexagons' in sizes:
    h = sizes['hexagons']
    print(f"Hexagons: {h['count']} windows")
    print(f"  Size range: {h['min']:.1f} - {h['max']:.1f} cm")
    print(f"  Average: {h['avg']:.1f} cm ({h['avg']/2.54:.1f} in)")

if 'pentagons' in sizes:
    p = sizes['pentagons']
    print(f"\nPentagons: {p['count']} windows")
    print(f"  Size range: {p['min']:.1f} - {p['max']:.1f} cm")
    print(f"  Average: {p['avg']:.1f} cm ({p['avg']/2.54:.1f} in)")


Hexagons: 81 windows
  Size range: 70.6 - 74.1 cm
  Average: 72.4 cm (28.5 in)

Pentagons: 8 windows
  Size range: 58.1 - 58.1 cm
  Average: 58.1 cm (22.9 in)


In [13]:
# Generate full assembly guide
guide = generate_assembly_guide(
    vertices=vertices,
    edges=edges,
    faces=faces,
    strut_width=chapel_info['strut_width']
)

# Format and print
guide_text = format_assembly_guide_text(
    guide,
    strut_width=chapel_info['strut_width'],
    strut_depth=chapel_info['strut_depth']
)

print(guide_text)


CHAPEL OF MOOP - ASSEMBLY GUIDE

STRUCTURE SUMMARY
----------------------------------------------------------------------
  Total Vertices:     198
  Total Struts:       286
  Total Windows:      89

STRUT DIMENSIONS
----------------------------------------------------------------------
  Cross-section:      5.1 x 5.1 cm (2.0" x 2.0")
  Type:               Wedged Struts (Radial Side Faces)

WINDOW OPENING SIZES (Resin Panel Dimensions)
----------------------------------------------------------------------
  Hexagons (81 total):
    Side-to-side: 65.6 - 69.0 cm
                  (25.8" - 27.2")
    Average:      67.3 cm (26.5")
  Pentagons (8 total):
    Side-to-side: 53.1 cm (20.9")

STRUT CUT LIST (Actual Cut Lengths)
----------------------------------------------------------------------
  37.3 cm (14.7"): 36 pieces
  41.4 cm (16.3"): 72 pieces
  42.2 cm (16.6"): 40 pieces
  42.6 cm (16.8"): 68 pieces
  42.7 cm (16.8"): 34 pieces
  44.4 cm (17.5"): 36 pieces



## Done!

The dome generator is working. Check the `output/` directory for the exported files:
- `dome_2v.step` / `dome_2v.stl` - Simple 2V test dome
- `chapel_dome_3v.step` / `chapel_dome_3v.stl` - Full Chapel of MOOP dome
